In [1]:
with open('/home/user/dxc/motion/motion-diffusion-model/dataset/HumanML3D/test.txt', 'r') as file:
    data = file.readlines()

index = []
for i in range (200):
    index.append(data[i].strip('\n'))

print(index)

['004822', '014457', '009613', '008463', '014160', '002530', '004945', '001969', '005799', '000749', '006658', '004124', '004965', '012805', '001168', '002246', '012506', '003424', '000787', '010665', '006518', '008399', '011997', '003245', '011978', '000021', '001783', '004488', '007543', '001486', '012657', '005213', '014477', '008382', '006652', '008664', '010962', '007742', '010940', '008292', '008131', '008320', '012498', '010915', '004388', '011441', '006701', '005188', '003784', '005064', '007561', '000818', '012883', '002139', '011673', '007767', '003566', '001632', '009958', '011684', '003005', '012309', '002606', '000742', '007862', '010720', '007450', '007995', '011988', '007779', '003450', '012529', '011028', '013812', '012361', '013207', '004040', '006521', '004394', '001567', '010819', '001448', '009867', '002104', '014351', '001888', '009377', '003111', '005278', '002627', '010792', '005197', '011213', '001059', '012143', '004311', '009161', '011363', '009539', '001120',

In [2]:
for i in index:
    with open("/home/user/dxc/motion/motion-diffusion-model/dataset/HumanML3D/texts/"+i+".txt","r") as f:
        lines = f.readlines()
        line = lines[0].split('#')[0]
        print(line)
        #写入到新的文件中
        with open("/home/user/dxc/motion/motion-diffusion-model/texts——motion.txt","a") as f:
            f.write(line+'\n')

a person is walking in place at a slow pace.
the person swings a golf club.
the man runs back wards
a man walks forward, then squats to pick something up with both hands, stands back up, and resumes walking.
a person waves with his right hand.
a person walks towards the camera.
person is bent down trying to pick up stuff, left arm is moved to the back, picks up more stuff and touches back again while bending down
a man walks forward, then turns around and walks back before facing back and standing still.
man walks forward while upper body is leaning slightly to the left and steps are unbalanced and slow.
a man using both hands to lift something off ground and places it back on ground in a slightly different position
the person walks backwards in a straight line
man walks forward moving hands and neck.
a person walks forward to the left, picks something up and walks back and then shakes what is in the hand.
a man crouches down while quickly walking forward and then stands up straight.
a

In [5]:
import numpy as np
from scipy.fftpack import dct, idct

def dct_low_pass_filter(data, axis=0, keep_ratio=0.05):
    """
    对输入矩阵进行 DCT 变换，保留低频信息，并重建矩阵。

    参数:
    - data: np.ndarray
      输入数据，形状为 [nframes, njoints, nfeats]。
    - axis: int, 默认 0
      指定进行 DCT 的轴（例如时间轴 nframes）。
    - keep_ratio: float, 默认 0.1
      保留的低频系数比例，范围为 (0, 1)。

    返回:
    - reconstructed_data: np.ndarray
      经过低频保留并重建的数据，形状与原始数据一致。
    - low_freq_dct: np.ndarray
      只保留低频信息的 DCT 系数矩阵。
    """
    # 确保 keep_ratio 合法
    if not (0 < keep_ratio <= 1):
        raise ValueError("keep_ratio 必须在 (0, 1] 范围内。")
    
    # 计算保留的频率数量
    n_elements = data.shape[axis]
    keep_freq = int(n_elements * keep_ratio)
    
    # 对指定轴进行 DCT
    dct_data = dct(data, axis=axis, norm='ortho')
    # 确保使用可迭代对象
    print(np.min(dct_data))  # 获取 dct_data 的最小值
    print(np.max(dct_data))  # 获取 dct_data 的最大值
        
    # 构造低频矩阵
    low_freq_dct = np.zeros_like(dct_data)
    slicer = [slice(None)] * data.ndim  # 构造切片
    slicer[axis] = slice(0, keep_freq)  # 设置低频范围
    low_freq_dct[tuple(slicer)] = dct_data[tuple(slicer)]
    
    # 逆 DCT 重建数据
    reconstructed_data = idct(low_freq_dct, axis=axis, norm='ortho')
    
    return reconstructed_data, low_freq_dct

In [4]:
import os
import numpy as np
from data_loaders.humanml.utils.paramUtil import t2m_kinematic_chain  # 骨架定义
from data_loaders.humanml.utils.plot_script import plot_3d_motion  # 动画生成函数
import torch
import matplotlib.pyplot as plt

def load_and_visualize_motion(npy_path, out_dir, text, fps=20):
    """
    从指定的 .npy 文件加载运动数据，并生成 3D 动画。
    """
    plt.figure(figsize=(10, 6))
    ax1 = plt.subplot(2, 1, 1)
    ax2 = plt.subplot(2, 1, 2)
    # 检查文件是否存在
    if not os.path.exists(npy_path):
        print(f"File not found: {npy_path}")
        return

    # 加载运动数据
    motion_data = np.load(npy_path)  # 假定形状为 [njoints, nfeats, nframes]
    #motion_data = motion_data.transpose(2, 0, 1)  # 转为 [nframes, njoints, nfeats]
    # 转换为 PyTorch 张量
    ax1.plot(motion_data[:, 15, 0])
    #设置标题
    ax1.set_title('Original Motion')
    
    #motion_data,_ = dct_low_pass_filter(motion_data)
    ax2.plot(motion_data[:, 15, 0])
    #设置标题
    ax2.set_title('Low Pass Motion')
    plt.savefig('./low_pass.png')
    plt.show()

    # 设置输出路径
    filename = os.path.basename(npy_path).replace('.npy', '.mp4')
    out_path = os.path.join(out_dir, filename)

    # 确保输出目录存在
    os.makedirs(out_dir, exist_ok=True)

    # 调用 plot_3d_motion 函数生成动画
    plot_3d_motion(
        save_path=out_path,          # 保存路径
        kinematic_tree=t2m_kinematic_chain,  # 骨架结构
        joints=motion_data,          # 运动数据 [nframes, njoints, nfeats]
        title=text,    # 动画标题
        dataset="humanml",           # 数据集名
        fps=fps                      # 帧率
    )

    print(f"Animation saved at: {out_path}")


# 示例调用
if __name__ == "__main__":
    # 修改为你的 .npy 文件路径
    npy_file = "/home/user/dxc/motion/motion-diffusion-model/00.npy"
    output_dir = "./animations"
    text = "Motion Animation"
    # 调用函数生成动画
    load_and_visualize_motion(npy_file, output_dir, text)

Animation saved at: ./animations/00.mp4


In [25]:
with open("/home/user/dxc/motion/motion-diffusion-model/texts——motion.txt","r") as f:
    lines = f.readlines()

for i in range(len(lines)):
    output_dir = './animations/gt_animation'
    text = lines[i]
    npy_file = '/home/user/dxc/motion/motion-diffusion-model/dataset/HumanML3D/new_joints/' + str(index[i]) + '.npy'
    print(text)
    load_and_visualize_motion(npy_file, output_dir, text)

a person is walking in place at a slow pace.

Animation saved at: ./animations/gt_animation/004822.mp4
the person swings a golf club.

Animation saved at: ./animations/gt_animation/014457.mp4
the man runs back wards

Animation saved at: ./animations/gt_animation/009613.mp4
a man walks forward, then squats to pick something up with both hands, stands back up, and resumes walking.

Animation saved at: ./animations/gt_animation/008463.mp4
a person waves with his right hand.

Animation saved at: ./animations/gt_animation/014160.mp4
a person walks towards the camera.

Animation saved at: ./animations/gt_animation/002530.mp4
person is bent down trying to pick up stuff, left arm is moved to the back, picks up more stuff and touches back again while bending down

Animation saved at: ./animations/gt_animation/004945.mp4
a man walks forward, then turns around and walks back before facing back and standing still.

Animation saved at: ./animations/gt_animation/001969.mp4
man walks forward while up

In [49]:
files = [f for f in os.listdir('/home/user/dxc/motion/StableMoFusion/checkpoints/t2m_condunet1d_2/samples_iter330000_seed0_texts——motion/joints_npy') if  f.endswith(".npy")]
with open("/home/user/dxc/motion/motion-diffusion-model/texts——motion.txt","r") as f:
    lines = f.readlines()

for file in files:
    i = int(file.split('.')[0])
    output_dir = './animations/mofusion_ours'
    text = lines[i]
    npy_file = '/home/user/dxc/motion/StableMoFusion/checkpoints/t2m_condunet1d_2/samples_iter330000_seed0_texts——motion/joints_npy/' + file
    load_and_visualize_motion(npy_file, output_dir, text)

Animation saved at: ./animations/mofusion_ours/82.mp4
Animation saved at: ./animations/mofusion_ours/66.mp4
Animation saved at: ./animations/mofusion_ours/137.mp4
Animation saved at: ./animations/mofusion_ours/93.mp4
Animation saved at: ./animations/mofusion_ours/60.mp4
Animation saved at: ./animations/mofusion_ours/51.mp4
Animation saved at: ./animations/mofusion_ours/104.mp4
Animation saved at: ./animations/mofusion_ours/85.mp4
Animation saved at: ./animations/mofusion_ours/63.mp4
Animation saved at: ./animations/mofusion_ours/13.mp4
Animation saved at: ./animations/mofusion_ours/07.mp4
Animation saved at: ./animations/mofusion_ours/74.mp4
Animation saved at: ./animations/mofusion_ours/177.mp4
Animation saved at: ./animations/mofusion_ours/89.mp4
Animation saved at: ./animations/mofusion_ours/158.mp4
Animation saved at: ./animations/mofusion_ours/166.mp4
Animation saved at: ./animations/mofusion_ours/109.mp4
Animation saved at: ./animations/mofusion_ours/26.mp4
Animation saved at: ./

In [50]:
import os
import shutil

# 源文件夹（当前文件夹）
source_folder = "/home/user/dxc/motion/motion-diffusion-model/animations/mofusion_ours"
# 目标文件夹
destination_folder = "/home/user/dxc/motion/motion-diffusion-model/animations/mofusion_base"

# 如果目标文件夹不存在，则创建
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# 获取当前文件夹下所有包含 "rep00" 的 .mp4 文件
files = [f for f in os.listdir(source_folder) if ".mp4" in f and f.endswith(".mp4")]

# 按字母顺序排序文件列表（可根据需求调整排序规则）
files.sort()

# 初始化计数器
counter = 0

# 遍历文件并移动到目标文件夹，同时重新编号
for file in files:
    # 新文件名，例如 rep00_001.mp4
    print(file)
    i = file.split('.mp4')[0]
    
    i = int(i)
    new_name = f"{index[i]}.mp4"
    source_path = os.path.join(source_folder, file)
    destination_path = os.path.join(destination_folder, new_name)
    
    # 移动文件并重命名
    shutil.move(source_path, destination_path)
    print(f"Moved: {file} -> {destination_path}")
    
    # 计数器加 1
    counter += 1

print(f"Done! Moved {len(files)} files to {destination_folder}.")

00.mp4
Moved: 00.mp4 -> /home/user/dxc/motion/motion-diffusion-model/animations/mofusion_base/004822.mp4
01.mp4
Moved: 01.mp4 -> /home/user/dxc/motion/motion-diffusion-model/animations/mofusion_base/014457.mp4
02.mp4
Moved: 02.mp4 -> /home/user/dxc/motion/motion-diffusion-model/animations/mofusion_base/009613.mp4
03.mp4
Moved: 03.mp4 -> /home/user/dxc/motion/motion-diffusion-model/animations/mofusion_base/008463.mp4
04.mp4
Moved: 04.mp4 -> /home/user/dxc/motion/motion-diffusion-model/animations/mofusion_base/014160.mp4
05.mp4
Moved: 05.mp4 -> /home/user/dxc/motion/motion-diffusion-model/animations/mofusion_base/002530.mp4
06.mp4
Moved: 06.mp4 -> /home/user/dxc/motion/motion-diffusion-model/animations/mofusion_base/004945.mp4
07.mp4
Moved: 07.mp4 -> /home/user/dxc/motion/motion-diffusion-model/animations/mofusion_base/001969.mp4
08.mp4
Moved: 08.mp4 -> /home/user/dxc/motion/motion-diffusion-model/animations/mofusion_base/005799.mp4
09.mp4
Moved: 09.mp4 -> /home/user/dxc/motion/motion-di

In [8]:
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def plot_single_frame(save_path, kinematic_tree, joints, title, frame_idx, dataset, figsize=(5, 5), radius=3, vis_mode='default', gt_frames=[]):
    """
    绘制单帧的3D运动图像并保存为图片。

    :param save_path: 图片保存路径
    :param kinematic_tree: 运动学树，用于定义关节连接关系
    :param joints: 关节位置数据，形状为 (帧数, 关节数, 3)
    :param title: 图片标题
    :param frame_idx: 要绘制的帧索引
    :param dataset: 数据集名称（影响缩放比例）
    :param figsize: 图片尺寸
    :param radius: 可视化范围
    :param vis_mode: 可视化模式（'default', 'upper_body', 'gt'）
    :param gt_frames: 包含 GT（真实值）帧索引的列表
    """
    # 数据预处理
    data = joints.copy().reshape(len(joints), -1, 3)
    if dataset == 'kit':
        data *= 0.003
    elif dataset == 'humanml':
        data *= 1.3
    elif dataset in ['humanact12', 'uestc']:
        data *= -1.5

    MINS = data.min(axis=0).min(axis=0)
    MAXS = data.max(axis=0).max(axis=0)

    height_offset = MINS[1]
    data[:, :, 1] -= height_offset

    trajec = data[:, 0, [0, 2]]
    data[..., 0] -= data[:, 0:1, 0]
    data[..., 2] -= data[:, 0:1, 2]

    colors_blue = ["#4D84AA", "#5B9965", "#61CEB9", "#34C1E2", "#80B79A"]  # GT color
    colors_orange = ["#DD5A37", "#D69E00", "#B75A39", "#FF6D00", "#DDB50E"]  # Generation color
    colors = colors_orange
    if vis_mode == 'upper_body':
        colors[0] = colors_blue[0]
        colors[1] = colors_blue[1]
    elif vis_mode == 'gt':
        colors = colors_blue

    # 绘制单帧
    fig = plt.figure(figsize=figsize)
    ax = Axes3D(fig)

    ax.set_xlim3d([-radius / 2, radius / 2])
    ax.set_ylim3d([0, radius])
    ax.set_zlim3d([-radius / 3., radius * 2 / 3.])
    ax.view_init(elev=120, azim=-90)
    ax.dist = 7.5

    used_colors = colors_blue if frame_idx in gt_frames else colors
    for i, (chain, color) in enumerate(zip(kinematic_tree, used_colors)):
        linewidth = 4.0 if i < 5 else 2.0
        ax.plot3D(data[frame_idx, chain, 0], data[frame_idx, chain, 1], data[frame_idx, chain, 2],
                  linewidth=linewidth, color=color)

    plt.axis('off')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    plt.title('\n'.join(title.split(' ')), fontsize=10)

    # 保存图片
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# 示例调用
# save_path = "frame.png"
# kinematic_tree = [[0, 1], [1, 2], [2, 3]]  # 关节连接关系示例
# joints = np.random.rand(50, 15, 3)  # 随机生成关节数据 (帧数, 关节数, 3)
# plot_single_frame(save_path, kinematic_tree, joints, "Example Frame", frame_idx=10, dataset='humanml')

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_motion_with_trajectory(save_path, kinematic_tree, joints, frame_indices, dataset, key_joints, figsize=(10, 10), radius=3):
    """
    绘制人物的移动轨迹和骨架动作在一张图片上。
    """

    # ------------------ 数据预处理 ------------------
    data = joints.copy().reshape(len(joints), -1, 3)
    
    # 不同数据集的缩放
    if dataset == 'kit':
        data *= 0.003
    elif dataset == 'humanml':
        data *= 1.3
    elif dataset in ['humanact12', 'uestc']:
        data *= -1.5

    # 将角色整体抬到 y>=0
    MINS = data.min(axis=0).min(axis=0)
    data[:, :, 1] -= MINS[1]

    # 记录根关节的 (x,z) 轨迹
    trajec = data[:, 0, [0, 2]]  # 根关节在每一帧的 x,z
    
    # ------ 这两行会把角色“钉”在 (0,0), 不要再减去根关节位置 ------
    # data[..., 0] -= data[:, 0:1, 0]
    # data[..., 2] -= data[:, 0:1, 2]

    # ------------------ 开始绘图 ------------------
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111, projection='3d')

    # 设置绘图范围（可根据需要调整）
    ax.set_xlim3d([-radius / 2, radius / 2])
    ax.set_ylim3d([0, radius])
    ax.set_zlim3d([-radius / 3., radius * 2 / 3.])
    ax.view_init(elev=120, azim=-90)
    ax.dist = 7.5

    # 绘制轨迹（蓝色虚线），注意此时 trajec 是真实全局的 (x,z)
    ax.plot(
        trajec[:, 0], 
        np.zeros_like(trajec[:, 0]), 
        trajec[:, 1],
        color='blue', 
        linestyle='--', 
        linewidth=1.5, 
        label='Trajectory'
    )

    # 逐帧绘制骨架
    for frame_idx in frame_indices:
        color = plt.cm.viridis(frame_idx / len(joints))  # 根据帧索引映射颜色
        
        # 绘制运动学树每条“骨骼链”
        for chain in kinematic_tree:
            ax.plot3D(
                data[frame_idx, chain, 0],
                data[frame_idx, chain, 1],
                data[frame_idx, chain, 2],
                linewidth=2,
                color=color
            )

        # 在关键关节绘制“球”
        for joint in key_joints:
            ax.scatter(
                data[frame_idx, joint, 0],
                data[frame_idx, joint, 1],
                data[frame_idx, joint, 2],
                color=color,
                s=50,
                edgecolor='k'
            )

    plt.legend()
    plt.axis('off')

    # 保存图片
    plt.savefig(save_path, dpi=300, bbox_inches='tight', transparent=False)
    plt.close()

# ------------------ 示例调用 ------------------
# import numpy as np
# save_path = "motion_with_trajectory.png"
# kinematic_tree = [[0, 1], [1, 2], [2, 3]]  # 关节连接关系示例
# joints = np.random.rand(50, 15, 3)         # 随机生成关节数据 (帧数, 关节数, 3)
# frame_indices = [0, 10, 20, 30, 40]        # 要绘制的帧索引
# key_joints = [0, 1, 3, 7, 10]             # 关键关节索引
# plot_motion_with_trajectory(save_path, kinematic_tree, joints, frame_indices, dataset='humanml', key_joints=key_joints)

In [25]:
def plot_single_frame(save_path, kinematic_tree, joints, title, frame_idx, dataset, key_joints, figsize=(5, 5), radius=3, vis_mode='default', gt_frames=[]):
    """
    绘制单帧的3D运动图像并保存为图片（仅展示关键关节点）。

    :param key_joints: 需要绘制圆球的关键关节索引列表，例如 [0, 1, 2, 3]
    """
    # 数据预处理
    data = joints.copy().reshape(len(joints), -1, 3)
    if dataset == 'kit':
        data *= 0.003
    elif dataset == 'humanml':
        data *= 1.3
    elif dataset in ['humanact12', 'uestc']:
        data *= -1.5

    MINS = data.min(axis=0).min(axis=0)
    data[:, :, 1] -= MINS[1]
    trajec = data[:, 0, [0, 2]]
    data[..., 0] -= data[:, 0:1, 0]
    data[..., 2] -= data[:, 0:1, 2]

    colors_blue = ["#4D84AA", "#5B9965", "#61CEB9", "#34C1E2", "#80B79A"]  # GT color
    colors_orange = ["#DD5A37", "#D69E00", "#B75A39", "#FF6D00", "#DDB50E"]  # Generation color
    colors = colors_orange
    if vis_mode == 'upper_body':
        colors[0] = colors_blue[0]
        colors[1] = colors_blue[1]
    elif vis_mode == 'gt':
        colors = colors_blue

    # 绘制单帧
    fig = plt.figure(figsize=figsize)
    ax = Axes3D(fig)

    ax.set_xlim3d([-radius / 2, radius / 2])
    ax.set_ylim3d([0, radius])
    ax.set_zlim3d([-radius / 3., radius * 2 / 3.])
    ax.view_init(elev=120, azim=-90)
    ax.dist = 7.5

    used_colors = colors_blue if frame_idx in gt_frames else colors
    for i, (chain, color) in enumerate(zip(kinematic_tree, used_colors)):
        linewidth = 4.0 if i < 5 else 2.0
        ax.plot3D(data[frame_idx, chain, 0], data[frame_idx, chain, 1], data[frame_idx, chain, 2],
                  linewidth=linewidth, color=color)

    # 在关键关节处绘制圆球
    for joint in key_joints:
        ax.scatter(data[frame_idx, joint, 0], data[frame_idx, joint, 1], data[frame_idx, joint, 2],
                   color='red', s=50, edgecolor='k')  # 使用红色标注关键点

    plt.axis('off')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    plt.title('\n'.join(title.split(' ')), fontsize=10)

    # 保存图片
    #plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.savefig(save_path, dpi=300, bbox_inches='tight', transparent=True)
    plt.close()

# 示例调用
# save_path = "frame_with_key_joints.png"
# kinematic_tree = [[0, 1], [1, 2], [2, 3]]  # 关节连接关系示例
# joints = np.random.rand(50, 15, 3)  # 随机生成关节数据 (帧数, 关节数, 3)
# key_joints = [0, 1, 3, 7, 10]  # 关键关节索引
# plot_single_frame(save_path, kinematic_tree, joints, "Frame with Key Joints", frame_idx=10, dataset='humanml', key_joints=key_joints)

In [62]:
import os
import numpy as np
from data_loaders.humanml.utils.paramUtil import t2m_kinematic_chain  # 骨架定义
#from data_loaders.humanml.utils.plot_script import plot_3d_motion  # 动画生成函数


def load_and_visualize_motion(npy_path, out_dir, text, fps=20):
    """
    从指定的 .npy 文件加载运动数据，并生成 3D 动画。
    """
    # 检查文件是否存在
    if not os.path.exists(npy_path):
        print(f"File not found: {npy_path}")
        return

    # 加载运动数据
    motion_data = np.load(npy_path)  # 假定形状为 [njoints, nfeats, nframes]
    #motion_data = motion_data.transpose(2, 0, 1)  # 转为 [nframes, njoints, nfeats]
    motion_data,_ = dct_low_pass_filter(motion_data,keep_ratio=0.03)
    # 设置输出路径
    filename = os.path.basename(npy_path).replace('.npy', '.png')
    out_path = os.path.join(out_dir, filename)

    # 确保输出目录存在
    os.makedirs(out_dir, exist_ok=True)

    # 调用 plot_3d_motion 函数生成动画
    plot_motion_with_trajectory(
        save_path=out_path,          # 保存路径
        kinematic_tree=t2m_kinematic_chain,  # 骨架结构
        joints=motion_data,          # 运动数据 [nframes, njoints, nfeats]
        
        frame_indices=[i*10 for i in range(10)], 
        dataset="humanml",
        key_joints= [0,9,15,4,5,10,11,20,21,16,17]           # 数据集名
                       # 帧率
    )
    print(f"Animation saved at: {out_path}")


# 示例调用
if __name__ == "__main__":
    # 修改为你的 .npy 文件路径
    npy_file = "/home/user/dxc/motion/motion-diffusion-model/dataset2/HumanML3D/new_joints/000027.npy"
    output_dir = "./animations"
    text = "Motion Animation"
    # 调用函数生成动画
    load_and_visualize_motion(npy_file, output_dir, text)

Animation saved at: ./animations/000027.png
